In [1]:
#検索用の行列を作成します,まずは行列を作る前にtextembeddingで取得したベクトルデータを小分けにして保存する
from create_embedding_vector_from_res import create_embedding_vector_from_res,encode_filename
import bs4
from bs4 import BeautifulSoup
import requests
import pickle,re,os
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

count = 0 #これが一定以上になったらpickleとしてセーブする
page_title_and_embedding_vectors_dict = {}
for page_title,data in info_for_search_dict.items():
    print(f"{count}:{page_title}")
    year = data["page_year"]
    with open(f"../../data_for_search/main_wiki_html/{year}/{encode_filename(page_title)}.pickle","rb") as f:
        res = pickle.load(f)
    soup = BeautifulSoup(res, 'html.parser')
    
    result =  create_embedding_vector_from_res(soup)
    page_title_and_embedding_vectors_dict[page_title] = {"overview_embedding":result[0],"story_embedding":result[1],"character_embedding":result[2]}
    count+=1
    if count %1000==0:
        with open(f"./embedding_vectors/embedding_vectors{count}.pickle","wb") as f:
            pickle.dump(page_title_and_embedding_vectors_dict,f)

#最後に余剰分を保存する
with open(f"./embedding_vectors/embedding_vectors{count}.pickle","wb") as f:
    pickle.dump(page_title_and_embedding_vectors_dict,f)
    page_title_and_embedding_vectors_dict = {}

0:クックとプッケ
1:スピード太郎
2:タンタンのコンゴ探険
3:ブロンディ (漫画)
4:タンタン アメリカへ
5:のらくろ
6:ファラオの葉巻
7:ファニーズ・オンパレード
8:冒険ダン吉
9:青い蓮
10:タンクタンクロー
11:フラッシュ・ゴードン
12:かけた耳
13:リトル・ルル
14:ザ・ファントム
15:フクちゃん
16:黒い島のひみつ
17:アクション・コミックス
18:オトカル王の杖
19:スーパーマン
20:金のはさみのカニ
21:グリーンランタン
22:ジャスティス・ソサエティ・オブ・アメリカ
23:仲よし手帖
24:バットマン (コミック)
25:フラッシュ (DCコミックス)
26:アクアマン
27:グリーンアロー
28:ふしぎな流れ星
29:プラスチックマン
30:翼賛一家大和さん
31:ワンダーウーマン
32:なぞのユニコーン号
33:サザエさん
34:マァチャンの日記帳
35:ヤネウラ3ちゃん
36:ロストワールド (漫画)
37:火星博士
38:新宝島
39:バット君
40:ムーミンまんがシリーズ
41:リル・フォークス
42:吸血魔団
43:大空魔王
44:地底国の怪人
45:ペ子ちゃん
46:妖怪探偵團
47:リツル・ペドロ
48:あんみつ姫
49:沙漠の魔王
50:スーパーボーイ
51:デンスケ (漫画)
52:似たもの一家
53:メトロポリス (漫画)
54:黄金都市
55:ジャングル大帝
56:ピーナッツ (漫画)
57:プーサン
58:ふしぎ旅行記
59:化石島
60:来るべき世界 (漫画)
61:クリちゃん
62:グンナイ君
63:新世界ルルー
64:新やじきた道中記
65:天使の玉ちゃん
66:ピピちゃん
67:わんぱくデニス
68:太平洋Xポイント
69:鉄腕アトム
70:ぼくのそんごくう
71:町子かぶき迷作集
72:リボンの騎士
73:リンゴ園の少女
74:ロック冒険記
75:スーパーデューパーマン
76:トンちゃん
77:UTOPIA 最後の世界大戦
78:赤胴鈴之助
79:地球の悪魔
80:犯人は誰だ!!
81:火の鳥 (漫画)
82:まっぴら君
83:空気男爵
84:白面夜叉
85:ぼんこちゃん
86:ロボット三等兵
87:アッちゃん
88:嵐をこえて
89:仙人部落
90:鉄人28号
91:ほのぼの

In [21]:
#前のコードでpage_title_and_embedding_vectors_dictを保存後に初期化するのを忘れていたため、数が増加するごとにめちゃくちゃ大きくなってしまっていた・・・これはよくない！
import os 
embedding_vectors_pickle_list = os.listdir("./embedding_vectors/")
embedding_vectors_pickle_dict = {int(embedding_vectors_pickle.replace(".pickle","").replace("embedding_vectors","")):embedding_vectors_pickle for embedding_vectors_pickle in embedding_vectors_pickle_list}
embedding_vectors_pickle_dict = sorted(embedding_vectors_pickle_dict.items())
embedding_vectors_pickle_list = list(y for x,y in embedding_vectors_pickle_dict) #必要なのはpickleの名前だけなのでそれを取り出す

full_page_title_list = []
for index, embedding_vectors_pickle  in enumerate(embedding_vectors_pickle_list):
    with open(f"./embedding_vectors/{embedding_vectors_pickle}","rb") as f:
        data = pickle.load(f)
    
    for page_title in full_page_title_list:
        if page_title in data:data.pop(page_title)
    
    for page_title in data:
        full_page_title_list.append(page_title)
    
    # for page_title in data:
    #     print(page_title)
    with open(f"./embedding_vectors/{embedding_vectors_pickle}","wb") as f:
        pickle.dump(data,f)


In [22]:
#保存したembedding_vectorsを元にmatrixを作ります(ベクトルから漫画の類似度を出力できるかテスト.ipynb　から拝借)

import os, json, pickle,re

# import numpy as torch
import torch
title_list = []
overview_embedding_matrix = torch.empty((0,512))
story_embedding_matrix = torch.empty((0,512))
character_embedding_matrix = torch.empty((0,512))
for embedding_pickle in os.listdir("./embedding_vectors/"):

    # if "2016" not in embedding_json: continue #2023年のデータだけとってくる。開発用・完成したらここはコメントアウト！

    with open("./embedding_vectors/" + embedding_pickle ,"rb") as f:
        pickle_file = pickle.load(f)
        number = int(embedding_pickle.replace(".pickle","").replace("embedding_vectors",""))
        print(f"{number}のベクトルデータを処理中...")
        for page_title,data in pickle_file.items():
            title = page_title
            title_list.append(title)
            overview_embedding = torch.tensor(data["overview_embedding"])
            story_embedding = torch.tensor(data["story_embedding"])
            character_embedding = torch.tensor(data["character_embedding"])

            overview_embedding_matrix = torch.vstack((overview_embedding_matrix,overview_embedding))
            story_embedding_matrix = torch.vstack((story_embedding_matrix,story_embedding))
            character_embedding_matrix = torch.vstack((character_embedding_matrix,character_embedding))
            
            
print(overview_embedding_matrix.shape)
print(story_embedding_matrix.shape)
print(character_embedding_matrix.shape)

embedding_matrix_list =[overview_embedding_matrix,story_embedding_matrix,character_embedding_matrix ]

c:\Users\taniguchizn.000\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1000のベクトルデータを処理中...
10000のベクトルデータを処理中...
11000のベクトルデータを処理中...
12000のベクトルデータを処理中...
13000のベクトルデータを処理中...
14000のベクトルデータを処理中...
15000のベクトルデータを処理中...
16000のベクトルデータを処理中...
17000のベクトルデータを処理中...
18000のベクトルデータを処理中...
19000のベクトルデータを処理中...
2000のベクトルデータを処理中...
20000のベクトルデータを処理中...
20132のベクトルデータを処理中...
3000のベクトルデータを処理中...
4000のベクトルデータを処理中...
5000のベクトルデータを処理中...
6000のベクトルデータを処理中...
7000のベクトルデータを処理中...
8000のベクトルデータを処理中...
9000のベクトルデータを処理中...
torch.Size([20132, 512])
torch.Size([20132, 512])
torch.Size([20132, 512])


In [ ]:
#matrixをhstackしてから保存しようとしたがfilesizeが100mbを超えたのでなくなく断念・・・そのため分割して保存します
import pickle
with open("../../data_for_search/title_and_embedding_matrix/title_list.pickle","wb") as f:
    pickle.dump(title_list,f)

with open("../../data_for_search/title_and_embedding_matrix/overview_embedding_matrix.pickle","wb") as f:
    pickle.dump(overview_embedding_matrix,f)

with open("../../data_for_search/title_and_embedding_matrix/story_embedding_matrix.pickle","wb") as f:
    pickle.dump(story_embedding_matrix,f)

with open("../../data_for_search/title_and_embedding_matrix/character_embedding_matrix .pickle","wb") as f:
    pickle.dump(character_embedding_matrix ,f)
